In [ ]:
import torch
from diffusers import StableDiffusionXLPipeline, StableDiffusionXLImg2ImgPipeline

In [ ]:
base_pipeline = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True,
)

base_pipeline = base_pipeline.to("cuda")
base_pipeline.enable_xformers_memory_efficient_attention()

In [ ]:
prompt = "Astronaut in a jungle, cold color palette, muted colors, detailed, 8k"
image = base_pipeline(prompt=prompt, output_type="latent").images[0]

In [ ]:
refiner_pipeline = StableDiffusionXLImg2ImgPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    text_encoder_2=base_pipeline.text_encoder_2,
    vae=base_pipeline.vae,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
)
refiner_pipeline = refiner_pipeline.to("cuda")
refiner_pipeline.enable_xformers_memory_efficient_attention()

In [ ]:
image = refiner_pipeline(prompt=prompt, image=image[None, :]).images[0]

In [ ]:
image